# Ax=b ? 

#### given a random matrix A and a vector b, we will try to find x st: Ax=b

In [3]:
import numpy as np
n=30
A = np.random.rand(n,n)
AS=A.T.dot(A)
#AS=A.T*A
b=np.ones(n)

### Exact solution

In [4]:
sol_ex=np.linalg.lstsq(A,b,rcond=None)[0]

In [5]:
x0=np.zeros(n)

### ARNOLDI PROCESS

In [6]:
def arnoldi_iteration(A,r0,M):
    n = A.shape[0]
    h = np.zeros((M + 1, M))
    V= np.zeros((n,M+1))
    V[:,0] = r0 / np.linalg.norm(r0)

    for j in range(M):
        w = A.dot(V[:,j])
        for i in range(j + 1):
            h[i, j] = np.dot(w,V[:,i])
            w = w - h[i, j] * V[:,i]

        h[j + 1, j] = np.linalg.norm(w)
        if h[j + 1, j] == 0:
            return V,h
        else:
            V[:,j+1] = w / h[j + 1, j]
    return V,h

In [7]:
r0=b-A.dot(x0)

In [8]:
V,h=arnoldi_iteration(A,r0,1)

array([[15.28308974],
       [ 1.56989857]])

### GMRES

In [7]:
# GMRES A BASE DE LA FONCTION ARNOLDI DEJA DEFINI
def GMRES_R(A, b, x0, nmax_iter,tol):
    m=min(nmax_iter, A.shape[0])
    n = A.shape[0]
    rm=b-A.dot(x0)
    for j in range(min(nmax_iter, A.shape[0])):
        beta=np.linalg.norm(rm)
        V,h=arnoldi_iteration(A,rm,j+1)
        s=np.zeros(j+2)
        s[0]=beta
        y=np.linalg.lstsq(h,s,rcond=None)[0]
        xm=x0+V[:,:-1].dot(y)
        rm=b-A.dot(xm)
        if np.linalg.norm(rm)<=tol:
            return xm
        else:
            x0=xm
    return xm

RK:<br>
j'ai defini m comme etant le minimum entre nombre d'iteration et la dimension de la matrie A

In [8]:
GMRES_R(A, b, x0,  n,10e-16)

array([-0.06575817,  0.33251067, -0.5729281 , -0.8131979 , -0.54688738,
       -0.30520559,  1.13005732,  0.16601282,  1.19785581, -1.1956725 ,
        0.07130517,  0.52358126,  0.78286441, -0.12647668,  0.07331804,
       -0.24769921, -0.83677894,  1.6340695 ,  0.75501444,  0.49200097,
       -0.14564908, -1.70791013,  1.48004496,  0.19575725,  0.07903655,
       -0.82935719, -1.21511526, -0.17379711, -0.00354716,  1.63978163])

In [9]:
sol_ex

array([-0.06575817,  0.33251067, -0.5729281 , -0.8131979 , -0.54688738,
       -0.30520559,  1.13005732,  0.16601282,  1.19785581, -1.1956725 ,
        0.07130517,  0.52358126,  0.78286441, -0.12647668,  0.07331804,
       -0.24769921, -0.83677894,  1.6340695 ,  0.75501444,  0.49200097,
       -0.14564908, -1.70791013,  1.48004496,  0.19575725,  0.07903655,
       -0.82935719, -1.21511526, -0.17379711, -0.00354716,  1.63978163])

### THE SYMMETRIC LANCZOS PROCESS

In [10]:
def SLP(A,v,m):
    n = A.shape[0]
    V= np.zeros((n,m+2))
    V[:,1]=v
    beta=np.zeros(m+2)
    alpha=np.zeros(m+2)
    for j in range(1,m+1):
        w = A.dot(V[:,j])-beta[j]*V[:,j-1]
        alpha[j]=np.dot(w,V[:,j])
        w=w-alpha[j]*V[:,j]
        beta[j+1]=np.linalg.norm(w)
        if beta[j+1]==0:
            return V,alpha,beta
        else:
            V[:,j+1]=w/beta[j+1]
    Tm=np.zeros([m,m])
    np.fill_diagonal(Tm,alpha[1:-1])
    np.fill_diagonal(Tm[1:,:-1],beta[2:-1])
    np.fill_diagonal(Tm[:-1,1:],beta[2:-1])
    return Tm,V[:,1:-1],alpha,beta
    

### Verification : Tm = Vm.TAVm

In [11]:
v = np.random.rand(n)
normalized_v = v/np.linalg.norm(v)

In [12]:
Tm,V,_,_=SLP(AS,normalized_v ,2)

In [13]:
Tm

array([[172.35430877,  93.27587224],
       [ 93.27587224,  53.09570218]])

In [14]:
V

array([[ 0.20262915,  0.08316072],
       [ 0.22143518, -0.02320922],
       [ 0.23084847,  0.05463803],
       [ 0.05292901,  0.25809707],
       [ 0.16494616,  0.00771284],
       [ 0.27767869, -0.07343001],
       [ 0.23406361, -0.05915094],
       [ 0.2804858 , -0.11373393],
       [ 0.03319008,  0.30429653],
       [ 0.1525191 ,  0.05486626],
       [ 0.03397658,  0.37057834],
       [ 0.1642521 ,  0.14814441],
       [ 0.19376546, -0.01276017],
       [ 0.08104869,  0.22439775],
       [ 0.25713403, -0.12192357],
       [ 0.24688544, -0.11235901],
       [ 0.11692914,  0.21686359],
       [ 0.14437791,  0.11877191],
       [ 0.04361118,  0.30455532],
       [ 0.17700046,  0.06904763],
       [ 0.15265439,  0.0471817 ],
       [ 0.09471126,  0.20384811],
       [ 0.18791243,  0.0539101 ],
       [ 0.04047128,  0.26298254],
       [ 0.18779279, -0.03558001],
       [ 0.29728957, -0.15696417],
       [ 0.27940205, -0.18724161],
       [ 0.0313564 ,  0.31830241],
       [ 0.22915426,

In [15]:
S=np.dot(V.T,np.dot(AS,V))
S

array([[172.35430877,  93.27587224],
       [ 93.27587224,  53.09570218]])

### THE SYMMETRIC LANCZOS ALG FOR LINEAR SYSTEMS

In [16]:
def SLLS(A, b, x0, nmax_iter,tol):
    n=A.shape
    rm=b-A.dot(x0)
    counter=0
    for m in range(min(nmax_iter, A.shape[0])):
        beta=np.linalg.norm(rm)
        v=rm / beta
        Tm,Vm,_,_=SLP(A,v,m+1)
        e1=np.zeros(m+1)
        e1[0]=1
        y=np.dot(beta*np.linalg.inv(Tm),e1)
        xm=x0+np.dot(Vm,y)
        rm=b-A.dot(xm)
        counter+=1
        if np.linalg.norm(rm)<=tol:
            return xm,counter
        else:
            x0=xm
    return xm,counter

In [17]:
AS=A.T.dot(A)

In [18]:
SLLS(AS,b,x0,n,10e-4)

(array([ -0.08267342,  -1.75615805,   5.24741237,  10.12015866,
          5.2904957 ,   5.88330149,  -6.78756383,   0.56783244,
        -12.28698972,  12.76901309,  -1.01982066,  -7.94097067,
         -3.88783706,   2.85798572,  -4.38634967,   3.97380469,
          8.37136648, -10.28334742,  -7.29027218,  -2.890096  ,
          0.6895806 ,  13.27831808, -14.55403273,  -1.55537226,
         -1.13266276,   6.80176358,  14.71077431,   3.84063388,
         -0.10675035, -15.0046158 ]),
 26)

In [19]:
sol_ex_sy=np.linalg.lstsq(AS,b,rcond=None)[0]
sol_ex_sy

array([ -0.08112645,  -1.75943472,   5.26886144,  10.15649825,
         5.31870338,   5.90673874,  -6.83539728,   0.56868664,
       -12.32833984,  12.81054416,  -1.01784308,  -7.95701485,
        -3.92171129,   2.85982466,  -4.38422581,   3.99231871,
         8.40492751, -10.33919565,  -7.3185995 ,  -2.90339488,
         0.68540822,  13.33973616, -14.60756679,  -1.56129672,
        -1.13837327,   6.83784288,  14.7674138 ,   3.85536289,
        -0.10508054, -15.0703437 ])

# CONSTAT:
pour le cas symmetrique, en definissant la matrice symmetrique par A.T.dot(A), l'algorithme converge en 2*n iterations avec n la taille de A

### THE NONSYMMETRIC LANCZOS BIORTHOGONALIZATION PROCESS

In [20]:
def NLBP(A,w,v,m):
    n=A.shape[0]
    #W=np.column_stack((np.zeros(n),w))
    #V=np.column_stack((np.zeros(n),v))
    V= np.zeros((n,m+2))
    V[:,1]=v
    W= np.zeros((n,m+2))
    W[:,1]=w
    beta=np.zeros(m+2)
    gama=np.zeros(m+2)
    alpha=np.zeros(m+2)
    add_last=np.zeros(m)
    for j in range(1,m+1):
        alpha[j]=np.dot(A.dot(V[:,j]),W[:,j])
        v=A.dot(V[:,j])-alpha[j]*V[:,j]-beta[j]*V[:,j-1]
        w=A.T.dot(W[:,j])-alpha[j]*W[:,j]-gama[j]*W[:,j-1]
        gama[j+1]=np.sqrt(np.abs(v.dot(w)))
        if gama[j+1]!=0:
            beta[j+1]=v.dot(w)/gama[j+1]
            W[:,j+1]=w/beta[j+1]
            V[:,j+1]=v/gama[j+1]
    Tm=np.zeros([m,m])
    add_last[-1]=gama[-1]
    Tm=np.row_stack((Tm,add_last))
    np.fill_diagonal(Tm,alpha[1:-1])
    np.fill_diagonal(Tm[1:,:-1],gama[2:-1])
    np.fill_diagonal(Tm[:-1,1:],beta[2:-1])
    return Tm,V[:,1:-1],W[:,1:-1]

In [21]:
v=np.random.rand(n)
w=np.zeros(n)
w[0]=1/v[0]

In [22]:
Tm,V,W=NLBP(A,w,v,2)

### Verification Tm=WmTAVm

In [23]:
Tm

array([[7.37475158, 7.28422295],
       [7.28422295, 7.8109757 ],
       [0.        , 1.66010703]])

In [24]:
W.T.dot(A.dot(V))

array([[7.37475158, 7.28422295],
       [7.28422295, 7.8109757 ]])

# THE BICONJUGATE GRADIENT (BCG) 

In [27]:
def BCG(A,b,x0,max_iter,tol):
    n=A.shape
    r0=b-A.dot(x0)
    p0=r0.copy()
    rt0=np.zeros(n)
    rt0[0]=1/r0[0]
    pt0=rt0.copy()
    counter=0
    while counter!=max_iter and np.linalg.norm(r0)>tol:
        alpha=r0.dot(rt0)/A.dot(p0).dot(pt0)
        x0=x0+alpha*p0
        r1=r0-alpha*A.dot(p0)
        rt1=rt0-alpha*A.T.dot(pt0)
        beta=r1.dot(rt1)/r0.dot(rt0)
        p1=r1+beta*p0
        pt1=rt1+beta*pt0
        r0=r1
        rt0=rt1
        p0=p1
        pt0=pt1
        counter+=1
    return x0,counter

In [65]:
BCG(A,b,x0,100,10e-4)

(array([-0.71232487, -0.15232019, -0.2480723 , -0.40939757,  0.02078813,
         1.01447317, -1.15060546, -1.08452049, -1.4795648 , -0.41534727,
         0.25618865,  1.29359381,  0.2837119 ,  0.95949902,  0.75405316,
         0.85554373,  0.36217283, -1.04461944,  1.50501175, -0.13241054,
         0.85753483, -1.02272784,  0.34989967,  0.31340192,  0.04951765,
         0.0582766 ,  0.03200622, -0.75200104,  0.9714533 , -0.50278576]),
 100)

In [29]:
sol_ex

array([-0.06575817,  0.33251067, -0.5729281 , -0.8131979 , -0.54688738,
       -0.30520559,  1.13005732,  0.16601282,  1.19785581, -1.1956725 ,
        0.07130517,  0.52358126,  0.78286441, -0.12647668,  0.07331804,
       -0.24769921, -0.83677894,  1.6340695 ,  0.75501444,  0.49200097,
       -0.14564908, -1.70791013,  1.48004496,  0.19575725,  0.07903655,
       -0.82935719, -1.21511526, -0.17379711, -0.00354716,  1.63978163])

### QUASI MINIMAL RESIDUAL (QMR) 

In [30]:
def QMR(A,b,x0,max_iter):
    r0=b-A.dot(x0)
    beta=np.linalg.norm(r0)
    w=r0/beta
    v=r0/beta
    for i in range(1,max_iter+1):
        Tm,V,_=NLBP(A,w,v,i+1)
        s=np.zeros(i+2)
        s[0]=beta
        y=np.linalg.lstsq(Tm,s,rcond=None)[0]
        xm=x0+V.dot(y)
    return xm
        
    

In [31]:
QMR(A,b,x0,2*n)

array([-0.06575817,  0.33251067, -0.5729281 , -0.8131979 , -0.54688738,
       -0.30520559,  1.13005732,  0.16601282,  1.19785581, -1.1956725 ,
        0.07130517,  0.52358126,  0.7828644 , -0.12647668,  0.07331804,
       -0.24769921, -0.83677894,  1.63406951,  0.75501444,  0.49200097,
       -0.14564908, -1.70791012,  1.48004496,  0.19575725,  0.07903655,
       -0.82935719, -1.21511526, -0.17379711, -0.00354716,  1.63978163])

In [32]:
sol_ex

array([-0.06575817,  0.33251067, -0.5729281 , -0.8131979 , -0.54688738,
       -0.30520559,  1.13005732,  0.16601282,  1.19785581, -1.1956725 ,
        0.07130517,  0.52358126,  0.78286441, -0.12647668,  0.07331804,
       -0.24769921, -0.83677894,  1.6340695 ,  0.75501444,  0.49200097,
       -0.14564908, -1.70791013,  1.48004496,  0.19575725,  0.07903655,
       -0.82935719, -1.21511526, -0.17379711, -0.00354716,  1.63978163])

### BICGSTAB

In [58]:
def BICGSTAB(A,b,x0,rt0,max_iter,tol):
    n=A.shape[0]
    r0=b-A.dot(x0)
    p0=b-A.dot(x0)
    counter=0
    while counter!=max_iter and np.linalg.norm(r0)>tol:
        alpha=r0.dot(rt0)/A.dot(p0).dot(rt0)
        s=r0-alpha*A.dot(p0)
        w=A.dot(s).dot(s)/np.dot(A.dot(s),A.dot(s))
        x0=x0+alpha*p0+w*s
        r1=s-w*(A.dot(s))
        beta=(r1.dot(rt0)/r0.dot(rt0))*alpha/w
        p1=r1+beta*(p0-w*A.dot(p0))
        r0=r1
        p0=p1
        counter+=1
    return x0,counter

In [59]:
rt0=np.random.rand(n)

In [62]:
BICGSTAB(A,b,x0,rt0,400,10e-12)

(array([-0.06575817,  0.33251067, -0.5729281 , -0.8131979 , -0.54688738,
        -0.30520559,  1.13005732,  0.16601282,  1.19785581, -1.1956725 ,
         0.07130517,  0.52358126,  0.78286441, -0.12647668,  0.07331804,
        -0.24769921, -0.83677894,  1.6340695 ,  0.75501444,  0.49200097,
        -0.14564908, -1.70791013,  1.48004496,  0.19575725,  0.07903655,
        -0.82935719, -1.21511526, -0.17379711, -0.00354716,  1.63978163]),
 208)

In [43]:
sol_ex

array([-0.06575817,  0.33251067, -0.5729281 , -0.8131979 , -0.54688738,
       -0.30520559,  1.13005732,  0.16601282,  1.19785581, -1.1956725 ,
        0.07130517,  0.52358126,  0.78286441, -0.12647668,  0.07331804,
       -0.24769921, -0.83677894,  1.6340695 ,  0.75501444,  0.49200097,
       -0.14564908, -1.70791013,  1.48004496,  0.19575725,  0.07903655,
       -0.82935719, -1.21511526, -0.17379711, -0.00354716,  1.63978163])

### GOLUB KAHAN BIDIAGOBALISATION

In [71]:
def GKB(A,u,max_iter):
    alpha=np.linalg.norm(A.T.dot(u))
    v=A.T.dot(u)/alpha
    U=np.zeros([n,max_iter])
    U[:,0]=u
    V=np.zeros([n,max_iter])
    V[:,0]=v
    for j in range(max_iter-1):
        U[:,j+1]=A.dot(V[:,j])-alpha*U[:,j]
        beta=np.linalg.norm(U[:,j+1])
        U[:,j+1]=U[:,j+1]/beta
        V[:,j+1]=A.T.dot(U[:,j+1])-beta*V[:,j]
        alpha=np.linalg.norm(V[:,j+1])
        V[:,j+1]=V[:,j+1]/alpha
    return U,V
        
        

In [72]:
e1=np.zeros(n)
e1[0]=1

In [73]:
U,V=GKB(A,e1,3)

array([[ 0.17685116,  0.1287993 , -0.00777384],
       [ 0.32878172, -0.18121267,  0.21577009],
       [ 0.33163416, -0.09810655, -0.28945425],
       [ 0.03527031,  0.26608841,  0.01445434],
       [ 0.12693343,  0.08042898, -0.26614434],
       [ 0.16459293,  0.12723031,  0.33163904],
       [ 0.06480777,  0.22750394,  0.24592992],
       [ 0.18788437,  0.05995414, -0.06319952],
       [ 0.0009993 ,  0.32679516,  0.24646207],
       [ 0.27046388, -0.13311787,  0.0827691 ],
       [ 0.03382685,  0.34083927, -0.08083464],
       [ 0.18568108,  0.10831678, -0.009207  ],
       [ 0.25227782, -0.09731564,  0.1653856 ],
       [ 0.0494878 ,  0.26118912, -0.16781919],
       [ 0.14976813,  0.07114698,  0.24846052],
       [ 0.23668948, -0.07134698,  0.01034884],
       [ 0.16717796,  0.12193468, -0.06716603],
       [ 0.23388424, -0.02916612, -0.11939931],
       [ 0.10276196,  0.18513321, -0.04162669],
       [ 0.0381822 ,  0.29939531, -0.39786427],
       [ 0.00461565,  0.2895454 ,  0.136